# Extra libraries annotation

The libraries were downloaded from https://littlemsandsailing.com/2023/10/free-ei-and-msms-spectra-in-nist-format/ and converted to SDF/MSP with the [LIB2NIST](https://chemdata.nist.gov/dokuwiki/doku.php?id=chemdata:nistlibs) tool.

In this notebook we annotate the spectra with SMILES strings. This information is taken from compound libraries, such as PubChem or IDSM. In order to do that we need a tool that conveniently communicates with the libraries - [MSMetaEnhancer](https://github.com/RECETOX/MSMetaEnhancer/tree/main). In the MSMetaEnhancer repository you'll find the installation instructions and instructions for setting up environment needed to run this notebook in. 

In [2]:
from matchms.importing import load_from_msp
from tqdm import tqdm
import pandas as pd
import asyncio
import nest_asyncio
from rdkit import Chem
from pathlib import Path

from MSMetaEnhancer import Application

In [3]:
DATA_ROOT_FOLDER = "../data"

## Helper functions

In [4]:
def annotate_spectra(original_msp_path,
                     annotated_msp_path,
                     services=['CTS', 'CIR', 'IDSM', 'PubChem', 'BridgeDb', 'RDKit'],
                     jobs=[('compound_name', 'canonical_smiles', 'PubChem'),
                           ('compound_name', 'canonical_smiles', 'IDSM')]):
    """
    Annotate the spectra in the original_msp_path and save the annotated spectra in the annotated_msp_path.
    You can specify the services you want to use for the annotation and the jobs you want to perform.
    The jobs are a list of tuple (X, Y, Z), meaning X_to_Y using service Z. The possible X and Y
    with respect to Z can be found in the documentation (code) of the MSMetaEnhancer.

    The code was taken and altered from the MSMetaEnhancer README example.
    """
    # Apply the nest_asyncio patch
    nest_asyncio.apply()
    app = Application()

    app.load_data(original_msp_path, file_format='msp')
    app.curate_metadata()
    asyncio.run(app.annotate_spectra(services, jobs))

    app.save_data(annotated_msp_path, file_format='msp')


def filter_unsuccessful_annotations(spectra,
                      smiles_column_name = "canonical_smiles",
                      check_from_smiles=[("formula", "formula"), ("molecular_weight", "mw")]):
    """
    Filter out spectra that have missing smiles, wrong formula or wrong molecular weight.
    You can specify the column name of the smiles and of the checks you want to perform.
    You can remove the check by removing the tuple from the check_from_smiles list.
    """
    bad_formula = 0
    bad_mw = 0
    missing_smiles = 0
    successful_spectra = []
    for s in spectra:
        smiles = s.metadata.get(smiles_column_name, None)
        if smiles == None:
            missing_smiles += 1
            continue

        ok = True
        for descriptor, column_name in check_from_smiles:
            desc_val = s.metadata.get(column_name, None)
            if desc_val == None:
                continue
            elif descriptor == "molecular_weight":
                mw_from_smiles = int(Chem.Descriptors.MolWt(Chem.MolFromSmiles(smiles)))
                if abs(int(desc_val) - mw_from_smiles) > 1:
                    bad_mw += 1
                    ok = False
                    print(f"Error in {s.metadata['compound_name']}, is {desc_val} but should be {mw_from_smiles}")
            elif descriptor == "formula":
                formula_from_smiles = Chem.rdMolDescriptors.CalcMolFormula(Chem.MolFromSmiles(smiles))
                if formula_from_smiles != desc_val:
                    bad_formula += 1
                    ok = False
                    print(f"Error in {s.metadata['compound_name']}, is {desc_val} but should be {formula_from_smiles}")
        if ok:
            successful_spectra.append(s)

    print(f"Missing smiles: {missing_smiles}")
    print(f"Bad formula: {bad_formula}")
    print(f"Bad mw: {bad_mw}")
    return successful_spectra


def remove_stereochemistry_and_canonicalize(smiles):
    """
    Remove the stereochemistry from a smiles and canonicalize it.
    --- copied from spectra_utils.py bcs of import issues ---
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Smiles {smiles} is not valid.")
        return None
    Chem.RemoveStereochemistry(mol)
    new_smiles = Chem.MolToSmiles(mol)
    return new_smiles


def extract_spectra(spectra):
    """
    Get the m/z, intensity and smiles from a list of spectra and
    return them in a DataFrame.
    --- copied from spectra_utils.py bcs of import issues ---
    """
    mzs = []
    intensities = []
    canon_smiless = []
    for d in tqdm(spectra):
        canon_smiles = remove_stereochemistry_and_canonicalize(d.metadata["canonical_smiles"])
        if canon_smiles:
            mzs.append(d.peaks.mz)
            intensities.append(d.peaks.intensities)
            canon_smiless.append(canon_smiles)
    df_out = pd.DataFrame({"mz": mzs, "intensity": intensities, "smiles": canon_smiless})
    return df_out

## SWGDRUG: Enhance the dataset metadata
Use MSMetaEnhancer to fill in the missing SMILES string. Afterward, filter out the spectra that do not have a SMILES string or were not annotated correctly.

In [12]:
library_name = "SWGDRUG_3"

data_path = f"{DATA_ROOT_FOLDER}/extra_libraries/{library_name}/{library_name}.MSP"
annotated_data_path = f"{DATA_ROOT_FOLDER}/extra_libraries/{library_name}/{library_name}_annotated.MSP"
output_jsonl = f"{DATA_ROOT_FOLDER}/extra_libraries/{library_name}/{library_name}.jsonl"

In [ ]:
annotate_spectra(data_path, annotated_data_path)

In [14]:
spectra_annotated = list(load_from_msp(annotated_data_path))

2024-11-06 09:50:52,605:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,607:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,607:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,608:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,609:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,610:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,611:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,612:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,613:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,614:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 09:50:52,616:WARNING:matchms:add_precursor_mz:No precursor_mz found i

In [15]:
clean_spectra_annotated = filter_unsuccessful_annotations(spectra_annotated)

Error in Clidinium, is C22H26NO3 but should be C22H26NO3+
Error in DDT, is 352 but should be 354
Error in Lindane, is 288 but should be 290
Error in Ambroxol, is 376 but should be 378
Error in Meperidine-d5, is C15H16D5NO2 but should be C15H21NO2
Error in Meperidine-d5, is 252 but should be 247
Error in PCP-d5, is C17H20D5N but should be C17H25N
Error in PCP-d5, is 248 but should be 243
Error in Alprazolam-d3, is C17H10D3ClN4 but should be C17H13ClN4
Error in Alprazolam-d3, is 311 but should be 308
Error in Diazepam-d3, is C16H10D3ClN2O but should be C16H13ClN2O
Error in Diazepam-d3, is 287 but should be 284
Error in Dextromethorphan-d3, is C18H22D3NO but should be C18H25NO
Error in Dextromethorphan-d3, is 274 but should be 271
Error in Nordiazepam-d5, is C15H6D5ClN2O but should be C15H11ClN2O
Error in Nordiazepam-d5, is 275 but should be 270
Error in Alprazolam-d8, is C17H5D8ClN4 but should be C17H13ClN4
Error in Alprazolam-d8, is 316 but should be 308
Error in Temazepam-d8, is C16H5D

In [16]:
print(f"Extracted {len(clean_spectra_annotated)}/{len(spectra_annotated)} spectra")

Extracted 3321/3598 spectra


### SWGDRUGS: Remove overlaps with NIST and SYNTH train set

In [17]:
canonical_swg_smiles = [remove_stereochemistry_and_canonicalize(s.metadata["canonical_smiles"]) for s in tqdm(clean_spectra_annotated)]

100%|██████████| 3321/3321 [00:00<00:00, 3996.11it/s]


In [18]:
## !!!!!!!!! THIS IS FOR DEUTERIUM HANDLING
spectra_annotated = clean_spectra_annotated

In [19]:
NIST_TRAIN_PATH = f'{DATA_ROOT_FOLDER}/nist/train.jsonl'
NIST_TEST_PATH = f'{DATA_ROOT_FOLDER}/nist/test.jsonl'
NIST_VAL_PATH = f'{DATA_ROOT_FOLDER}/nist/valid.jsonl'

NEIMS_TRAIN_PATH = f'{DATA_ROOT_FOLDER}/synth/neims_gen/train.jsonl'

In [20]:
nist_train = pd.read_json(NIST_TRAIN_PATH, lines=True, orient='records')
nist_test = pd.read_json(NIST_TEST_PATH, lines=True, orient='records')
nist_valid = pd.read_json(NIST_VAL_PATH, lines=True, orient='records')

neims_train = pd.read_json(NEIMS_TRAIN_PATH, lines=True, orient='records')

In [21]:
nist_train_set = set(nist_train.smiles)
nist_test_set = set(nist_test.smiles)
nist_valid_set = set(nist_valid.smiles)
neims_train_set = set(neims_train.smiles)
canonical_swg_smiles_set = set(canonical_swg_smiles)

notallowed_smiles_set = nist_train_set.union(neims_train_set)
allowed_smiles_set = canonical_swg_smiles_set.difference(notallowed_smiles_set)

print(f"overlap of swg and nist train: {len(canonical_swg_smiles_set.intersection(nist_train_set))}")
print(f"overlap of swg and nist test: {len(canonical_swg_smiles_set.intersection(nist_test_set))}")
print(f"overlap of swg and nist valid: {len(canonical_swg_smiles_set.intersection(nist_valid_set))}")
print(f"overlap of swg and neims train: {len(canonical_swg_smiles_set.intersection(neims_train_set))}")

print(f"unique swg: {len(canonical_swg_smiles_set)}")

print(f"unique swg without leaks: {len(allowed_smiles_set)}")

overlap of swg and nist train: 1557
overlap of swg and nist test: 191
overlap of swg and nist valid: 189
overlap of swg and neims train: 11
unique swg: 3197
unique swg without leaks: 1629


In [22]:
allowed_spectra = []
for s in tqdm(spectra_annotated):
    if remove_stereochemistry_and_canonicalize(s.metadata["canonical_smiles"]) in allowed_smiles_set:
        allowed_spectra.append(s)

100%|██████████| 3321/3321 [00:00<00:00, 3914.23it/s]


In [23]:
len(allowed_spectra)

1679

In [24]:
df = extract_spectra(allowed_spectra)
df["mz"] = df["mz"].apply(lambda x: list(x.round(0)))
# path_jsonl = Path(data_path).with_suffix(".jsonl")   # correct
path_jsonl = Path(data_path).parent / Path(Path(data_path).stem + "_noD.jsonl")  # test noD
df.to_json(path_jsonl, orient="records", lines=True)

100%|██████████| 1679/1679 [00:00<00:00, 3237.57it/s]


---------------------------------------------------------------------------

## Use this pipeline for all the other libraries

In [5]:
library_name = "bb"

data_path = f"../data/extra_libraries/{library_name}/{library_name}.MSP"
annotated_data_path = f"../data/extra_libraries/{library_name}/{library_name}_annotated.MSP"
output_jsonl = f"../data/extra_libraries/{library_name}/{library_name}.jsonl"

In [6]:
# ~ 9mins for Cayman_library
# ~ 88 min for MONA_GCMS
annotate_spectra(data_path, annotated_data_path,
                 jobs=[('compound_name', 'canonical_smiles', 'PubChem'),
                       ('compound_name', 'canonical_smiles', 'IDSM'),
                       ('inchikey', 'canonical_smiles', 'PubChem')])

2024-11-06 13:01:50,226:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,227:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,228:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,229:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,230:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,231:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,231:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,232:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,233:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,234:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 13:01:50,235:WARNING:matchms:add_precursor_mz:No precursor_mz found i

[13:02:09] WARNING: not removing hydrogen atom without neighbors
[13:02:09] WARNING: not removing hydrogen atom without neighbors
[13:02:09] WARNING: not removing hydrogen atom without neighbors
[13:02:12] WARNING: not removing hydrogen atom without neighbors
[13:58:49] Explicit valence for atom # 0 Cl, 7, is greater than permitted
[13:58:49] ERROR: Explicit valence for atom # 0 Cl, 7, is greater than permitted

[13:58:49] Explicit valence for atom # 1 Cl, 7, is greater than permitted
[13:58:49] Explicit valence for atom # 1 Cl, 7, is greater than permitted
[13:58:50] Explicit valence for atom # 1 Cl, 7, is greater than permitted
[13:58:50] Explicit valence for atom # 0 Cl, 7, is greater than permitted
[13:58:50] ERROR: Explicit valence for atom # 0 Cl, 7, is greater than permitted

[13:58:50] Explicit valence for atom # 1 Cl, 7, is greater than permitted


2024-11-06 14:07:40,534:WARNING:matchms:save_as_msp:Spectrum(s) will be stored as msp file with extension .MSP


In [7]:
spectra_annotated = list(load_from_msp(annotated_data_path))

2024-11-06 14:07:41,806:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,808:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,809:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,810:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,810:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,811:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,812:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,813:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,814:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,814:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-11-06 14:07:41,815:WARNING:matchms:add_precursor_mz:No precursor_mz found i

In [8]:
clean_spectra_annotated = filter_unsuccessful_annotations(spectra_annotated)

Error in DEUTERIUM, is D2 but should be H2
Error in DEUTERIUM, is 4 but should be 2
Error in METHYL SULFATE, is C2H6O4S but should be CH4O4S
Error in METHYL SULFATE, is 126 but should be 112
Error in METHYL SULFATE, is C2H6O4S but should be CH4O4S
Error in METHYL SULFATE, is 126 but should be 112
Error in DIMETHYLMERCURY, is 232 but should be 230
Error in HYDROGEN BROMIDE, is BrH but should be HBr
Error in DIBORANE, is B2H6 but should be H6B2
Error in ACETYLENE-D2, is C2D2 but should be C2H2
Error in ACETYLENE-D2, is 28 but should be 26
Error in VINYLCYCLOHEXENE DIOXIDE, is C8H10O2 but should be C8H12O2
Error in VINYLCYCLOHEXENE DIOXIDE, is 138 but should be 140
Error in GAMMA-1,2,3,4,5,6-HEXACHLOROCYCLOHEXANE, is 288 but should be 290
Error in DIETHYLMERCURY, is 260 but should be 258
Error in ETHANE-D, is C2H5D but should be C2H6
Error in ETHANE-1,2-D2, is C2H4D2 but should be C2H6
Error in ETHANE-1,2-D2, is 32 but should be 30
Error in ETHANE-1,1,1-D3, is C2H3D3 but should be C2H6
Er

[14:07:50] WARNING: not removing hydrogen atom without neighbors
[14:07:50] WARNING: not removing hydrogen atom without neighbors
[14:07:50] WARNING: not removing hydrogen atom without neighbors
[14:07:50] WARNING: not removing hydrogen atom without neighbors


Error in ETHANE-D5, is C2HD5 but should be C2H6
Error in ETHANE-D5, is 35 but should be 30
Error in ETHANE-D6, is C2D6 but should be C2H6
Error in ETHANE-D6, is 36 but should be 30
Error in DIFLUORAMINE, is F2HN but should be HF2N
Error in HYDROCHLORIC ACID, is ClH but should be HCl
Error in HYDROGEN CHLORIDE, is ClH but should be HCl
Error in FLUOROMETHANE-D3, is CD3F but should be CH3F
Error in FLUOROMETHANE-D3, is 37 but should be 34
Error in METHYLALLYLAMINE, is 111 but should be 71
Error in 1,3-DIOXOCANE, is 114 but should be 116
Error in 1,1,1,9-TETRACHLORONONANE, is 264 but should be 266
Error in 1,3-DIBROMOPROPANE, is 204 but should be 201
Error in 1,2-DIBROMOPROPANE, is 204 but should be 201
Error in DIPENTENE OXIDE, is C10H16O but should be C10H16O2
Error in DIPENTENE OXIDE, is 152 but should be 168
Error in PROPENE-1,1-D2, is C3H4D2 but should be C3H6
Error in PROPENE-1,1-D2, is 44 but should be 42
Error in TRIACETONE PEROXIDE, is C9H8O6 but should be C3H8O4
Error in TRIACET

Error in 1,2-DIBROMO-1,2-DICHLOROETHANE, is 254 but should be 256
Error in ACC. NO. 246785, is 336 but should be 338
Error in 1,1,3,3,4,4-HEXACHLOROBUT-1-ENE, is 260 but should be 262
Error in PENTACHLOROPROPENE, is 212 but should be 214
Error in HEXACHLOROBUTENE, is 260 but should be 262
Error in ACC. NO. 092891, is 212 but should be 214
Error in HEXACHLOROCYCLOHEXANE, is 288 but should be 290
Error in 2-METHYL-1H-BENZO(DE)QUINOLINE, is C12H10N2 but should be C13H11N
Error in PHENAZINE, is C12H10N2 but should be C12H8N2
Error in PHENAZINE, is 182 but should be 180
Error in OCTAHYDROPHENANTHRENE, is C14H18 but should be C14H24
Error in OCTAHYDROPHENANTHRENE, is 186 but should be 192
Error in ACC. NO. 165682, is C21H16 but should be C15H8N2O2
Error in ACC. NO. 165682, is 268 but should be 248
Error in TRIBROMOFLUOROMETHANE, is 268 but should be 270
Error in PENTACHLOROBUTADIENE, is 224 but should be 226
Error in MOLYBDENUM HEXAFLUORIDE, is 212 but should be 209
Error in ACC. NO. 108608,

In [9]:
print(f"Extracted {len(clean_spectra_annotated)}/{len(spectra_annotated)} spectra")

Extracted 6890/13634 spectra


### Remove overlaps with NIST and SYNTH train set

In [10]:
canonical_lib_smiles = [remove_stereochemistry_and_canonicalize(s.metadata["canonical_smiles"]) for s in tqdm(clean_spectra_annotated)]

100%|██████████| 6890/6890 [00:01<00:00, 5664.69it/s]


In [11]:
## !!!!!!!!! THIS IS FOR DEUTERIUM HANDLING
spectra_annotated = clean_spectra_annotated

In [12]:
NIST_TRAIN_PATH = f'{DATA_ROOT_FOLDER}/nist/train.jsonl'
NIST_TEST_PATH = f'{DATA_ROOT_FOLDER}/nist/test.jsonl'
NIST_VAL_PATH = f'{DATA_ROOT_FOLDER}/nist/valid.jsonl'

NEIMS_TRAIN_PATH = f'{DATA_ROOT_FOLDER}/synth/neims_gen/train.jsonl'

In [13]:
nist_train = pd.read_json(NIST_TRAIN_PATH, lines=True, orient='records')
nist_test = pd.read_json(NIST_TEST_PATH, lines=True, orient='records')
nist_valid = pd.read_json(NIST_VAL_PATH, lines=True, orient='records')

neims_train = pd.read_json(NEIMS_TRAIN_PATH, lines=True, orient='records')

In [14]:
nist_train_set = set(nist_train.smiles)
nist_test_set = set(nist_test.smiles)
nist_valid_set = set(nist_valid.smiles)
neims_train_set = set(neims_train.smiles)
canonical_lib_smiles_set = set(canonical_lib_smiles)

notallowed_smiles_set = nist_train_set.union(neims_train_set).union(neims_train_set)
allowed_smiles_set = canonical_lib_smiles_set.difference(notallowed_smiles_set)

print(f"overlap of lib and nist train: {len(canonical_lib_smiles_set.intersection(nist_train_set))}")
print(f"overlap of lib and nist test: {len(canonical_lib_smiles_set.intersection(nist_test_set))}")
print(f"overlap of lib and nist valid: {len(canonical_lib_smiles_set.intersection(nist_valid_set))}")
print(f"overlap of lib and neims train: {len(canonical_lib_smiles_set.intersection(neims_train_set))}")

print(f"unique lib: {len(canonical_lib_smiles_set)}")

print(f"unique lib without leaks: {len(allowed_smiles_set)}")

overlap of lib and nist train: 3704
overlap of lib and nist test: 510
overlap of lib and nist valid: 468
overlap of lib and neims train: 3
unique lib: 5203
unique lib without leaks: 1496


In [15]:
allowed_spectra = []
for s in tqdm(spectra_annotated):
    if remove_stereochemistry_and_canonicalize(s.metadata["canonical_smiles"]) in allowed_smiles_set:
        allowed_spectra.append(s)

100%|██████████| 6890/6890 [00:01<00:00, 5481.38it/s]


In [16]:
len(allowed_spectra)

1848

In [17]:
df = extract_spectra(allowed_spectra)
df["mz"] = df["mz"].apply(lambda x: list(x.round(0)))
# path_jsonl = Path(data_path).with_suffix(".jsonl")   # correct
path_jsonl = Path(data_path).parent / Path(Path(data_path).stem + "_noD.jsonl")  # test noD
df.to_json(path_jsonl, orient="records", lines=True)

100%|██████████| 1848/1848 [00:01<00:00, 1456.35it/s]


## playground

In [ ]:
# ssmaz
from rdkit import Chem

Chem.MolFromSmiles("C[C@]12CC[C@@H](C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4([C@H]3CCC4=O)C)O")

In [ ]:
mask = nist_train.smiles.apply(lambda x: "D" in x)
nist_train[mask]